### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime

# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-28 20:49:45


In [3]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"Obsidian path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
Obsidian path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A5\Data\name-ttl.csv


In [7]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [8]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\price-hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [9]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [10]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [11]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  08:49 PM


In [12]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.31,1.24,1.31,1.23,1.52,3.97,2025-02-28
ADVANC,276.00,273.00,278.00,193.00,307.00,-1.78,2025-02-28
AEONTS,108.50,104.50,108.50,103.50,168.50,1.4,2025-02-28
AH,13.50,13.50,14.10,13.30,24.80,-0.74,2025-02-28
AIE,1.60,1.51,1.61,0.72,1.80,0.0,2025-02-28
AIMIRT,10.60,10.50,10.70,10.20,11.20,0.0,2025-02-28
AIT,4.48,4.38,4.50,3.74,5.05,0.0,2025-02-28
AJ,2.18,2.16,2.28,2.24,7.15,-3.54,2025-02-28
AMATA,23.50,22.90,23.60,20.10,31.00,-1.67,2025-02-28
ANAN,0.43,0.41,0.43,0.40,0.94,0.0,2025-02-28
AOT,43.25,42.00,43.25,41.50,67.50,-0.57,2025-02-28
AP,8.95,8.60,8.95,7.50,11.20,2.87,2025-02-28
ASIAN,7.50,7.45,7.70,6.35,11.20,-2.6,2025-02-28
ASK,7.00,6.85,7.05,7.00,19.20,-4.76,2025-02-28
ASP,2.24,2.24,2.26,2.12,2.84,-0.88,2025-02-28
ASW,6.80,6.70,6.90,6.40,8.30,-1.45,2025-02-28
AWC,2.92,2.84,3.02,2.88,4.58,-5.81,2025-02-28
BA,20.20,19.80,20.60,15.40,26.00,0.0,2025-02-28
BAM,6.35,6.20,6.45,5.35,10.40,-0.78,2025-02-28
BANPU,4.14,4.04,4.18,4.12,7.50,0.0,2025-02-28
BAY,22.90,22.60,22.90,22.20,29.00,0.

In [13]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  08:55 PM


In [14]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
25,BCT,-,0.00,0.00,53.00,76.75,0.0,2025-02-28


In [15]:
out_df.shape

(221, 8)

In [16]:
out_df = out_df[out_df['name'] != 'BCT']

In [17]:
out_df = out_df[out_df['name'] != 'LPF']

In [18]:
out_df = out_df[out_df['name'] != 'TYCN']

In [19]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [20]:
out_df = out_df[out_df['name'] != 'MST']

In [21]:
out_df = out_df[out_df['name'] != 'SPC']

In [22]:
out_df.shape

(216, 8)

In [23]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [24]:
sorted_out_df = out_df.sort_values('percent')

In [25]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
78,HANA,16.10,15.90,16.80,20.10,49.75,-19.90,2025-02-28
134,PSH,4.80,4.80,5.60,5.50,12.50,-14.29,2025-02-28
186,THG,11.00,11.00,12.80,12.40,46.75,-13.39,2025-02-28
53,DCC,1.23,1.20,1.38,1.32,2.16,-8.21,2025-02-28
113,MAJOR,11.60,11.50,12.50,11.70,16.00,-7.94,2025-02-28
200,TQM,15.30,15.30,16.40,16.00,33.25,-7.27,2025-02-28
96,KCE,18.30,17.00,18.80,19.50,47.75,-7.11,2025-02-28
70,GFPT,8.15,8.15,8.75,8.20,13.40,-6.86,2025-02-28
147,RS,0.68,0.65,0.73,0.72,7.35,-6.85,2025-02-28
202,TSE,0.42,0.38,0.44,0.44,1.85,-6.67,2025-02-28


In [26]:
sorted_out_df.query('name == "MCS"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
116,MCS,7.1,7.05,7.15,5.8,8.2,-0.7,2025-02-28


In [27]:
out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
7,AJ,2.18,2.16,2.28,2.24,7.15,-3.54,2025-02-28
13,ASK,7.00,6.85,7.05,7.00,19.20,-4.76,2025-02-28
16,AWC,2.92,2.84,3.02,2.88,4.58,-5.81,2025-02-28
19,BANPU,4.14,4.04,4.18,4.12,7.50,0.00,2025-02-28
28,BEAUTY,0.21,0.20,0.22,0.21,0.73,0.00,2025-02-28
32,BGRIM,11.20,11.00,11.50,11.30,30.00,-3.45,2025-02-28
38,BPP,7.05,6.90,7.30,7.30,15.20,-3.42,2025-02-28
43,CHG,2.16,1.71,2.16,2.04,3.08,-4.42,2025-02-28
44,CK,13.80,11.60,14.00,13.90,24.60,-1.43,2025-02-28
49,CPN,49.25,47.75,49.75,48.75,68.75,-2.48,2025-02-28


In [28]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at


In [29]:
out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at


In [30]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [31]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\Price-Hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [32]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [33]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-28 20:55:02
